# Creación de un Agente IA con RAG para realizar recomendaciones si es buena idea viajar hacia algun lugar en la fecha mencionada.

### Paredes Jimmy

En esta Google Notebook, se encuentra la creacion de un Agente IA turisrtico, el cual dependiendo de la fecha y lugar que se proporcione nos mencionará si es recomendable viajar hacia el lugar.

## Instalar e importar la librerias necesarias

### Instalando librerias utilizadas

In [60]:
!pip install groq

In [61]:
!pip install duckduckgo-search --upgrade
!pip install trafilatura

In [62]:
!pip install gradio --quiet

In [63]:
!pip install python-dotenv

### Importanto librerias a utilizar

In [64]:
import os
from dotenv import load_dotenv

In [65]:
from groq import Groq

from sentence_transformers import SentenceTransformer
model = SentenceTransformer("all-MiniLM-L6-v2")

from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

import requests
from datetime import datetime
import trafilatura
from duckduckgo_search import DDGS

import gradio as gr

## Funciones para la construccion del agente IA



In [66]:
def obtener_lugar_fecha(contenido, api_key_groq):
    """ Esta función, obtiene el lugar y la fecha de un texto dado usando un LLM.
    Args:
      contenido: str
        La pregunta realizada por el usuario
      api_key_groq: str
        La API key de Groq
    Returns:
      lugar_usuario: str
        El lugar que el usuario desea visitar
      fecha_usuario: str
        La fecha en la cual el usuario desea visitar el lugar mencionado
    """

    anio_actual = datetime.now().year
    contenido += f"""Del texto dado, retornar unicamente el nombre del tema principal del cual se desea tener informacion
    y la fecha en Formato: YYYY-MM-DD, de la forma: tema_principal, fechas. Si el año no esta presente agrear el año {anio_actual}"""
    client = Groq(api_key=api_key_groq)
    completion = client.chat.completions.create(
      #model="compound-beta",
      model = "llama-3.3-70b-versatile",
        messages=[
            {
                "role": "user",
                'content': contenido
            }
        ]
    )
    lugar_usuario, fecha_usuario = completion.choices[0].message.content.split(", ")
    return lugar_usuario, fecha_usuario


In [67]:
def obtener_info_web(tema):
    """ Esta función, realiza una busqueda en la web sobre un tema dado y lo retorna.
    args:
        tema: str
            Tema a buscar en la web
    returns:
        results_str: str
            Texto completo de cada página encontrada
    """

    # Realizacion de la búsqueda
    results = []
    results_str = str()
    with DDGS() as ddgs:
        results = list(ddgs.text(tema, max_results=5))

    # Extraccion del texto completo de cada página
    for result in results:
        url = result["href"]
        try:
            downloaded = trafilatura.fetch_url(url)
            if downloaded:
                text = trafilatura.extract(downloaded)

                results_str += text + "\n"
        except Exception as e:
            continue
    return results_str


In [68]:
def split_text(text, max_length=300):
    """ Esta funcion divide un texto en partes de un tamaño maximo dado.
    args:
        text: str
            Texto a dividir
        max_length: int
            Longitud maxima de cada parte del texto
    returns:
        chunks: list
            Lista de partes del texto
    """

    sentences = text.split('. ')
    chunks, current = [], ''
    for sentence in sentences:
        if len(current) + len(sentence) < max_length:
            current += sentence + '. '
        else:
            chunks.append(current.strip())
            current = sentence + '. '
    chunks.append(current.strip())
    return chunks


In [69]:
def obtener_coordenadas_con_fallback(lugar):
    """ Esta funcion obtiene las coordenadas de latitud y longitud de un lugar dado usando la API de opencagedata.com
    args:
        lugar: str
            Nombre del lugar a buscar
    returns:
        coords: tuple
            Coordenadas de latitud y longitud del lugar
    """

    palabras = lugar.split()  # Separar por palabras
    intentos = []

    # Crear combinaciones de palabras progresivamente más cortas
    for i in range(len(palabras)):
        intentos.append(" ".join(palabras[i:len(palabras)]))  # Combinación progresiva más corta

    # Intentar obtener coordenadas con cada variante del lugar
    for intento in intentos:
        url = f"https://api.opencagedata.com/geocode/v1/json?q={intento}&key={API_OPENCAGE_KEY}&language=es"
        respuesta = requests.get(url).json()
        if respuesta["results"]:
            coords = respuesta["results"][0]["geometry"]
            return coords["lat"], coords["lng"]

    raise Exception("No se pudo obtener la ubicación.")



def obtener_clima(lat, lon):
    """ Esta funcion obtiene la informacion de clima del lugar dado en sus coordenadas usando la API de Tomorrow.io.
    args:
        lat: float
            Latitud del lugar
        lon: float
            Longitud del lugar
    returns:
        response: dict
            Informacion de clima del lugar
    """

    url = f"https://api.tomorrow.io/v4/weather/forecast?location={lat},{lon}&apikey={API_TOMORROW_KEY}&units=metric"
    response = requests.get(url).json()
    return response



def buscar_por_fecha(data, fecha_str):
    """ Esta funcion busca la informacion de clima del lugar dado en la fecha dada.
    args:
        data: dict
            Informacion de clima del lugar
        fecha_str: str
            Fecha a buscar en formato YYYY-MM-DD
    returns:
        dia: dict
            Informacion de clima del dia dado
    """

    fecha_deseada = datetime.strptime(fecha_str, "%Y-%m-%d").date()
    for dia in data["timelines"]["daily"]:
        fecha_actual = datetime.fromisoformat(dia["time"]).date()
        if fecha_actual == fecha_deseada:
            return dia
    return None


In [70]:
def obtener_respuesta(contenido_proc, api_key_groq):
    """ Esta funcion obtiene la respuesta del LLM para una pregunta dada, y a partir de la informacion adicional proveida
    args:
        contenido_proc: str
            String con informacion del lugar, informacion mas relevante obtenida de la web, clima y pregunta
        api_key_groq: str
            API key de Groq
    returns:
        respuesta: str
            Respuesta del LLM
    """

    client = Groq(api_key=api_key_groq)
    completion = client.chat.completions.create(
      #model="compound-beta",
      model = "llama-3.3-70b-versatile",
        messages=[
            {
                "role": "user",
                'content': contenido_proc
            }
        ]
    )

    return completion.choices[0].message.content


In [71]:
def obtener_recomendacion(pregunta, state):
    """ Esta funcion obtiene la recomendacion del agente turistico para una pregunta dada.
    args:
        pregunta: str
            Pregunta dada por el usuario
        state: dict
            Estado del agente
    returns:
        respuesta: str
            Respuesta del agente
    """

    # Crear estado si no existe
    if state is None:
        state = {}

    if "lugar_usuario" not in state or "fecha_usuario" not in state:
        lugar_usuario, fecha_usuario = obtener_lugar_fecha(pregunta, API_KEY)
        state["lugar_usuario"] = lugar_usuario
        state["fecha_usuario"] = fecha_usuario
    else:
        lugar_usuario = state["lugar_usuario"]
        fecha_usuario = state["fecha_usuario"]

    lugar_pregunta = lugar_usuario + pregunta

    info_web = obtener_info_web(lugar_pregunta)
    chunks = split_text(info_web, max_length=100)

    # Se crean los embedings usando los chunks resultantes de la informacion obtenida de la web
    embeddings = model.encode(chunks)

    # Se hace los embedings a la pregunta dada por el usuario
    pregunta_embedding = model.encode(pregunta)

    # Se obtienen los scores de los embedings de la informacion de la web
    scores = cosine_similarity([pregunta_embedding], embeddings)[0]

    # Se guardan los embedings que mas similitud tengan con la pregunta realizada por el usuario
    top_indices = np.argsort(scores)[::-1][:5]
    top_chunks = [chunks[i] for i in top_indices]
    state["top_chunks"] = top_chunks

    try:
        lat, lon = obtener_coordenadas_con_fallback(lugar_usuario)
        datos = obtener_clima(lat, lon)
        info_clima = buscar_por_fecha(datos, fecha_usuario)

        # Se realiza un cambio desde el diccionario obtenido del clima hacia string para agregarlo como informacion
        info_clima_texto = "\n\n".join([f"{clave}: {valor}" for clave, valor in info_clima.items()]) if info_clima else "No hay datos de clima disponibles."
    except Exception as e:
        info_clima_texto = f"Error al obtener el clima: {e}"

    # Se junta la informacion acerca del lugr, los chunks con mas similitud a la pregunta dada por el usuario, el clima
    # que presentara el lugar en la fecha dada y la pregunta realizada por el usuario
    contenido_proc = (
        "Información útil del lugar:\n"
        + "\n\n".join(top_chunks)
        + "\n\n" + info_clima_texto
        + f"\n\nPregunta del usuario: {pregunta}"
    )

    respuesta = obtener_respuesta(contenido_proc, API_KEY)
    return respuesta, state


## Carga de las API key desde el archivo .env

In [72]:
# Cargar las variables desde .env
load_dotenv()

# Acceder a las API keys
API_KEY = os.getenv("API_KEY") # API key de la aplicacion web groq para el uso de LLMs
API_TOMORROW_KEY = os.getenv("API_TOMORROW_KEY") # API de la aplicacion web tomorrow.io para obtener informacion del clima
API_OPENCAGE_KEY = os.getenv("API_OPENCAGE_KEY") # API de la aplicacion web opencagedata.com para obtener coordenadas de un lugar dado

# Interfaz de usuario usando Gradio

Aqui se crea una interfaz de usuario usando gradio para una mejor interacción con el agente creado

In [74]:
with gr.Blocks() as demo:
    estado = gr.State()

    gr.Markdown("## Agente Turístico")
    gr.Markdown("Haz una pregunta como: '¿Es recomendable viajar a la laguna de Yambo el dia 29 de abril?'")

    input_box = gr.Textbox(label="Tu pregunta")
    output_box = gr.Textbox(label="Respuesta")

    enviar_btn = gr.Button("Enviar")

    enviar_btn.click(
        obtener_recomendacion,
        inputs=[input_box, estado],
        outputs=[output_box, estado]
    )

demo.launch(share=True)
#demo.launch(share=True, debug=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c339bcfdda9aaba44d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
